<a href="https://colab.research.google.com/github/sixth-group/project2/blob/main/VisionWithTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download train data

In [ ]:
!gdown 1-7ljOXuzdIeXTCRuxZFGH2Sg3IanH-WJ
# https://drive.google.com/file/d/1-7ljOXuzdIeXTCRuxZFGH2Sg3IanH-WJ/view?usp=sharing
!unzip train_data.zip
%rm train_data.zip

In [5]:
rm -rf train_data/

In [16]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

from torchvision.models import resnet50

# Set up the directory paths
train_data_dir = '/content/train_data'
input_size = (224, 224)

# Define data transformations
train_transform = transforms.Compose([
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=20, translate=(0.2, 0.2)),
    transforms.RandomRotation(10),  # Rotate the image by +/- 10 degrees
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),  # Randomly change the brightness, contrast, and saturation
    transforms.Resize(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Create datasets
train_dataset = ImageFolder(train_data_dir, transform=train_transform)
dataset_size = len(train_dataset)

# Define the size of your training set, for example, 80% of the dataset
train_size = int(0.8 * dataset_size)

# The rest of the data will be used for validation
valid_size = dataset_size - train_size

# Split your dataset
train_dataset, valid_dataset = random_split(train_dataset, [train_size, valid_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)

# Load the pre-trained ResNet50 model
base_model = resnet50(pretrained=True)

# Replace the top layer of the pre-trained model
num_classes = 10
base_model.fc = nn.Linear(base_model.fc.in_features, num_classes)

# Define the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model = base_model.to(device)

# Define the optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        train_correct += (predicted == labels).sum().item()

    train_loss /= len(train_dataset)
    train_acc = train_correct / len(train_dataset)

    model.eval()
    valid_loss = 0.0
    valid_correct = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * images.size(0)
            valid_correct += (predicted == labels).sum().item()

    valid_loss /= len(valid_dataset)
    valid_acc = valid_correct / len(valid_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}: "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

Epoch 1/10: Train Loss: 2.0165, Train Acc: 0.2904, Valid Loss: 2.4832, Valid Acc: 0.2432
Epoch 2/10: Train Loss: 1.8161, Train Acc: 0.3569, Valid Loss: 2.0692, Valid Acc: 0.3013
Epoch 3/10: Train Loss: 1.7411, Train Acc: 0.3877, Valid Loss: 2.3281, Valid Acc: 0.2713
Epoch 4/10: Train Loss: 1.6474, Train Acc: 0.4233, Valid Loss: 1.8417, Valid Acc: 0.3624
Epoch 5/10: Train Loss: 1.5954, Train Acc: 0.4498, Valid Loss: 1.8879, Valid Acc: 0.3664
Epoch 6/10: Train Loss: 1.5224, Train Acc: 0.4729, Valid Loss: 1.8189, Valid Acc: 0.3834
Epoch 7/10: Train Loss: 1.4719, Train Acc: 0.4928, Valid Loss: 1.7182, Valid Acc: 0.4074
Epoch 8/10: Train Loss: 1.4289, Train Acc: 0.5062, Valid Loss: 1.6449, Valid Acc: 0.4545
Epoch 9/10: Train Loss: 1.3752, Train Acc: 0.5211, Valid Loss: 1.6396, Valid Acc: 0.4404
Epoch 10/10: Train Loss: 1.3266, Train Acc: 0.5444, Valid Loss: 1.7974, Valid Acc: 0.3984


In [ ]:
!gdown 1-5xCKnZ7N7X6m0pBV1yZDoTmJjAQpttF
# https://drive.google.com/file/d/1-5xCKnZ7N7X6m0pBV1yZDoTmJjAQpttF/view?usp=sharing
!unzip test_data.zip
%rm test_data.zip

In [ ]:
# rm -rf test_data/

In [ ]:
# Assuming that test_data_dir is the directory where your test data is located
test_data_dir = '/content/test_data'

# Create an ImageDataGenerator for the test data
test_data_generator = ImageDataGenerator()

# Use flow_from_directory to generate batches of test data
test_generator = test_data_generator.flow_from_directory(
    test_data_dir,
    target_size=input_size,
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)

# Use the model to predict the labels of the test data
predictions = model.predict(test_generator)

# The predictions are probabilities for each class. To convert these probabilities into actual class predictions, you can use argmax
predicted_classes = np.argmax(predictions, axis=1)


In [ ]:
import pandas as pd

# Get the filenames from the generator
filenames = test_generator.filenames

# Create a DataFrame with filenames and predicted classes
df = pd.DataFrame({
    'name': filenames,
    'predicted': predicted_classes
})

# Save the DataFrame to a CSV file
df.to_csv('q1_submission.csv', index=False)
